#### Recap `plane` wave and `spherical` wave

`Plane` wave

$$\boxed{e^{i \textbf{k} \cdot \textbf{r}}=e^{i(k_x x +k_y y +k_z z)}}$$

* Any $\textbf{r}$ having a `constant` dot-product with $\textbf{k}$ (i.e., constant phase) lies on a line/plane perpendicular to $\textbf{k}$
* As a result, for time-dependent expression $e^{i (\textbf{k} \cdot \textbf{r} - \omega t)}$, $\textbf{k}$ is the `propagation direction` and line/plane of constant phase is `wavefront`
* `Wave number` $|\textbf{k}|=\frac{2\pi}{\lambda}$, and `phase velocity` $c=\frac{\omega}{|\textbf{k}|}$

`Spherical` wave

$$\boxed{U(\mathbf{r})=\frac{e^{i k |\textbf{r}|}}{4\pi|\mathbf{r}|}}$$

* All $\textbf{r}$ having a `constant` distance from origin form a wavefront (i.e., a sphere)
* The time-dependent expression, with wavenumber $k$ and angular frequency $\omega$, is

$$U(\mathbf{r},t)=\frac{e^{i k |\textbf{r}|}}{4\pi|\mathbf{r}|}e^{-i\omega t}$$

* `Wave number` $k=\frac{2\pi}{\lambda}$, and `phase velocity` $c=\frac{\omega}{k}$, here, there's no need to specifify $k$ as a vector as the propagation direction is either $\mathbf{r}$ (for source) or $-\mathbf{r}$ (for sink)

#### `Huygens-Fresnel` principle

Every point on a wavefront is itself the source of spherical waves, and the secondary waves emanating from different points mutually interfere

According to Huygens principle, the `spherical wave eminating from each point on the wavefront` has two additional coefficients

$$U(\mathbf{r})=\boxed{-\frac{i}{\lambda} K(\chi)} \frac{e^{i k |\textbf{r}|}}{4\pi|\mathbf{r}|}$$

or

$$U(\mathbf{r})=\boxed{\frac{1}{i\lambda} K(\chi)} \frac{e^{i k |\textbf{r}|}}{4\pi|\mathbf{r}|}$$

where $\chi$ is the angle between the normal of the primary wavefront and the normal of the secondary wavefront


$K(\chi)=\cos(\chi)$ is generally used, meaning the secondary wave decays as its wavefront normal becomes less aligned with primary wavefront normal

$-\frac{i}{\lambda}$ indicates the secondary wave is $\frac{\pi}{2}$ out of phase, as

$$e^{i(\phi-\frac{\pi}{2})}=e^{i\phi}e^{-i\frac{\pi}{2}}=-ie^{i\phi}=\frac{1}{i}e^{i\phi}$$

So

$$\boxed{U(\mathbf{r})=\frac{1}{\lambda} \cos(\chi) \frac{e^{i (k |\textbf{r}|-\frac{\pi}{2})}}{4\pi|\mathbf{r}|}}$$

In [ ]:
import sympy as smp
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Arc, Rectangle, Ellipse, Circle
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
np.random.seed(42)
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})
plt.style.use('dark_background')

In [ ]:
class HuygensPrinciple:
    def __init__(self, kx, ky, omega, dt, duration):
        if kx == 0 and ky==0:
            raise ValueError('k must be non-zero')

        self.kx = kx
        self.ky = ky
        self.k = np.sqrt(self.kx**2 + self.ky**2)
        self.omega = omega
        self.dt = dt
        self.duration = duration

        self.x = np.linspace(-3*np.pi, 3*np.pi, 300)
        self.y = np.linspace(-3*np.pi, 3*np.pi, 300)
        self.xx, self.yy = np.meshgrid(self.x, self.y)

        self.wavelength = 2*np.pi/self.k

    def compute_cosine_angle(self):
        # compute cosine coefficient for U(r)
        self.k_vector = np.array([self.kx, self.ky])
        self.grid_vectors = np.stack((self.xx, self.yy), axis=-1)
        self.dot_product = np.dot(self.grid_vectors, self.k_vector)

        self.k_magnitude = np.linalg.norm(self.k_vector)
        self.grid_magnitudes = np.linalg.norm(self.grid_vectors, axis=-1)

        self.cosine_angle = self.dot_product / (self.k_magnitude * self.grid_magnitudes)

        return self.cosine_angle

    def plot_wave(self, time):
        """handles the actual wave plotting at a given time"""
        # compute distance r at each grid
        self.abs_r = np.sqrt(self.xx**2 + self.yy**2)

        # compute wave value at each grid
        self.plane_amp_clip = 0.08  # clip amplitude of plane wave to better visualize
        self.plane_wave = self.plane_amp_clip*(np.exp(1j * (self.kx * self.xx + self.ky * self.yy - self.omega * time))).real
        self.spherical_wave = self.cosine_angle_grid * (1/(4*np.pi*self.abs_r) * np.exp(1j * (self.k * self.abs_r - self.omega * time - np.pi/2))).real # 1/lambda omitted in coefficient

        self.plane_wave[self.xx > 0] = 0
        self.spherical_wave[self.xx <= 0] = 0
        self.wave = self.plane_wave + self.spherical_wave

        self.ax.clear()

        self.level_clip = 0.2  # clip colorbar range to better visualize
        self.levels = self.level_clip*np.linspace(-1/(4*np.pi*np.min(self.abs_r)), 1/(4*np.pi*np.min(self.abs_r)), 300)
        self.c = self.ax.contourf(self.xx, self.yy, self.wave, levels=self.levels, cmap='seismic')
        if not hasattr(self, 'colorbar'):  # Only add colorbar once
            self.colorbar = self.fig.colorbar(self.c, ax=self.ax, label='Wave Amplitude')

        self.ax.plot([0,0], [self.y.min(),-0.3], 'k', linewidth=2) # interface
        self.ax.plot([0,0], [0.3,self.y.max()], 'k', linewidth=2) # interface

        self.ax.set_aspect('equal', adjustable='box')
        self.ax.set_xlabel('x')
        self.ax.set_ylabel('y')
        self.ax.set_title(f'$\Delta t$: {time:.2f} s, $\omega$: {self.omega} rad/s, $k_x$: {self.kx}, $k_y$: {self.ky} \n temporal phase shift: {self.omega*time/(2*np.pi):.2f} cycle(s)')

        self.ax.quiver(0, 0, self.wavelength, 0, angles='xy', scale_units='xy', scale=1, color='r', width=0.01, alpha=0.8, label='$\lambda$')
        self.ax.legend()
        # self.ax.grid(True)
        self.fig.tight_layout()

    def main(self, animation=False):
        """main method to initialize plot and decides whether to animate or just show static plot."""
        self.animation = animation
        self.cosine_angle_grid = self.compute_cosine_angle()
        self.fig, self.ax = plt.subplots(figsize=(6, 5)) # this only needs setup once
        self.plot_wave(0) # plot at time 0 if not otherwise specified

        if self.animation:
            return self.animate() # this calls self.animate() and returns HTML object
        else:
            plt.show()

    def update(self, frame):
        """used for updating the plot in each frame during the animation"""
        self.plot_wave(frame)

    def animate(self):
        """set up and return the animation"""
        self.num_frames = int(self.duration / self.dt)
        self.ani = FuncAnimation(self.fig, self.update, frames=np.linspace(0, self.duration, self.num_frames), blit=False)
        plt.close(self.fig)
        return HTML(self.ani.to_jshtml())

In [ ]:
slit_diffraction = HuygensPrinciple(kx=1, ky=0, omega=1, dt=0.25, duration=4*np.pi)
slit_diffraction.main(animation=True)